In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.081, 0.075, 0.105, 0.101],
    [0.058, 0.073, 0.087, 0.093],
    [0.052, 0.087, 0.073, 0.09],
    [0.039, 0.079, 0.081, 0.08],
    [0.080, 0.077, 0.239, 0.071],
    [0.278, 0.058, 0.117, 0.062],
    [0.038, 0.123, 0.015, 0.131],
    [0.028, 0.089, 0.018, 0.134],
    [0.036, 0.119, 0.108, 0.095],
    [0.098, 0.093, 0.121, 0.081],
    [0.013, 0.127, 0.038, 0.063]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.15288888, 0.30888675, 0.45773133, 0.080493041])

# NORMALIZATION FUNCTION (min-max normalization)
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# CODAS METHOD
def codas(matrix, weights, criteria_types, sites, criteria, tau=0.02):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Negative-ideal solution (worst for each criterion)
    neg_ideal = np.min(weighted_matrix, axis=0)

    # Euclidean and Taxicab distances from negative-ideal
    euclidean = np.sqrt(((weighted_matrix - neg_ideal) ** 2).sum(axis=1))
    taxicab = np.abs(weighted_matrix - neg_ideal).sum(axis=1)

    # Assessment score
    # If difference in Euclidean distance between two alternatives < tau, use Taxicab distance as tie-breaker
    scores = np.zeros(len(sites))
    for i in range(len(sites)):
        count = 0
        for j in range(len(sites)):
            if i == j:
                continue
            diff = euclidean[i] - euclidean[j]
            if diff > tau:
                count += 1
            elif abs(diff) <= tau:
                if taxicab[i] > taxicab[j]:
                    count += 1
        scores[i] = count

    # Alternatively, for ranking, you can use the Euclidean distance directly (higher is better)
    df = pd.DataFrame({
        'Euclidean': euclidean,
        'Taxicab': taxicab,
        'CODAS Score': euclidean,  # For simple ranking, use Euclidean distance
        'Rank': rankdata(-euclidean, method='min')
    }, index=sites).sort_values('Rank')

    return df

result_df = codas(matrix, weights, criteria_types, sites, criteria)
print(result_df)


        Euclidean   Taxicab  CODAS Score  Rank
Site5    0.467277  0.591504     0.467277     1
Site9    0.334995  0.513276     0.334995     2
Site11   0.312444  0.357004     0.312444     3
Site7    0.301377  0.382543     0.301377     4
Site10   0.272622  0.443568     0.272622     5
Site6    0.258493  0.361320     0.258493     6
Site1    0.207496  0.342845     0.207496     7
Site3    0.179963  0.302145     0.179963     8
Site2    0.167424  0.274896     0.167424     9
Site4    0.166303  0.264000     0.166303    10
Site8    0.160780  0.234053     0.160780    11
